In [1]:
%%capture --no-display

%pip install geopandas
%pip install pyrosm
%pip install gtfs_kit
%pip install h3
%pip install pyproj
%pip install keplergl
%pip install matplotlib
%pip install esy-osmfilter
%pip install rasterio
%pip install joypy
%pip install sklearn
%pip install pyproj

# Data aquisition and preprocessing for new danish Transit Acessibility score

In this notebook we go through the different steps taken in order to produce the raw data needed for generating the transit accesibility map for Denmark as visualized here https://potatotvnet.github.io/transit/

The project utilizes data from a variety of different sources. These are listed here for completeness
* We downloaded the entire schedule of the public transit in Denmark for a typical week in march 2022 (20/03/22 to 26/03/22) in the standard GTFS data format using the Rejseplanen API https://help.rejseplanen.dk/hc/da/articles/214174465-Rejseplanens-API (danish only). This data range does not include any special holidays.
* We used OpenStreetMap data for the entirety of Denmark as collected by Geofabrik.de (https://download.geofabrik.de/europe/denmark.html). The data is used for generating the walkable paths that connect people to public transit.
* We downloaded the Global Human Settlement Layers (GHSL) covering all of Denmark (https://ghsl.jrc.ec.europa.eu/download.php?ds=smod). The layers are satelite images and were used to determine the rural and urban areas of Denmark. 

In terms of Python libraries that work on geospatial data we highligt the following tools
* H3 was used to convert points, stops and paths from the WGS 84 coordinate reference system to a spatial index of hexagons to decrease the amount of space and computation needed in order to work with this much data
* GTFS kit provides powerful functionality to convert data from the GTFS format to more Python standard formats like the geodataframe from geopandas.
* Geopandas/pandas standard tools for data wrangling
* esy-osmfilter Loading large pbf files (such as the danish OSM data) using libraries such as Pyrosm which converts the data to a GeoDataFrame is very memory intensive and should be avoided when working on a standard laptop. We use esy-osmfilter to filter the OSM data to include only what we want and save it to a geojson which we can work with in python.
* Rasterio provides functionality to work with raster images such as satelite images. We use it for reading, rendering and projecting the GHSL data.

In [27]:
import sys
from pathlib import Path
import json
import h3
import pandas as pd
import numpy as np
import geopandas as gp
import shapely as shp
import gtfs_kit as gk
from shapely.geometry import mapping, shape

## Read GTFS feeds
Initially we wanted to see if our methods would scale to include more than one transit ressource (ie multiple GTFS files) however we later found this to be out of scope for the project

Specify the paths to the zip files here. 

In [28]:
H3_RES = 11

In [29]:
# read GTFS data from multiple GTFS files is possible
paths = ["../resources/rejseplanen.zip"]
feeds = []
for path in paths:
    feeds.append(gk.read_feed(path, dist_units='km'))

dates = ['20220320', '20220321', '20220322', '20220323', '20220324', '20220325', '20220326']

Map all stops to their corresponding H3 cell

In [30]:
# map all stops to h3 for each feed (ie each GTFS file)
stop_h3_maps = []
for feed in feeds:
    feed.stops['h3'] = feed.stops.apply(lambda row: h3.geo_to_h3(row['stop_lat'],row['stop_lon'],H3_RES), axis=1)
    stop_h3_maps.append(pd.Series(feed.stops.h3.values,index=feed.stops.stop_id).to_dict())


Write back data in geojson

In [6]:
%%capture --no-display
geojson = feed.routes_to_geojson(include_stops=True)
with open("data/geojson.json", "w") as outfile:
    json.dump(geojson, outfile)

## Work covering the danish transit data

Take the danish transit feed and turn the routes into a GeoDataFrame

In [ ]:
%%capture --no-display
routes = gk.routes.geometrize_routes(feeds[0])
routes.head()


Transform geometries into arrays containing geometry points and write data

In [ ]:
def transform_geo(geo):
    if type(geo) == shp.geometry.LineString:
        return [[g[0], g[1]] for g in list(geo.coords)]
    else:
        return [[g[0], g[1]] for g in list(geo.geoms[0].coords)]

routes_filtered = routes
routes_new = []

for idx, route in routes_filtered.iterrows():
    name = route["route_short_name"]
    category = route["route_type"]
    geo = route["geometry"]
    if type(geo) == shp.geometry.LineString:
        routes_new.append([name, category, [[g[0], g[1]] for g in list(geo.coords)]])
    else:
        for ge in geo.geoms:
            routes_new.append([name, category, [[g[0], g[1]] for g in list(ge.coords)]])

routes_df = pd.DataFrame(routes_new, columns= ['name', 'category', 'geo'])

routes_df.head()

In [9]:
routes_df.to_csv("data/routes.csv")

### Calculate Frequencies
We compute a time series for every stop in our one week interval for every 1 hour

In [31]:
# compute time series for the Rejseplan schedule this includes the frequency of departures at every stop, every hour for a week.
ts = []
for feed in feeds:
    a = feed.compute_stop_time_series(dates=dates, freq='60Min').transpose().reset_index(level=1)
    ts.append(a)

Implement a function that takes a list of lists and adds elements

In [32]:
def add_lists(lists):
    res = np.array([0]*len(lists[0]))
    for i in range(len(lists)):
        l = np.array(lists[i])
        l = np.nan_to_num(l)
        # print(f"adding res with {np.shape(res)} and list with {np.shape(l)}")
        res = np.add(res, l)
        # print(f"resulting shape: {np.shape(res)}")
    return res

Here we create our initial h3 map which maps the frequency of departures as a sum of all departures during an hour within our specified hexagons. This means the value of a hexagon can be the sum of departures from multiple stops. We furthermore create a queue of stops. 

In [33]:
result_df = pd.DataFrame()
for idx, feed in enumerate(feeds):
    tts = ts[idx]
    result_df['stop_id'] = tts['stop_id'].values.tolist()
    result_df['h3'] = result_df.apply(lambda row: stop_h3_maps[idx][row['stop_id']], axis=1)
    result_df['frequency'] = tts.iloc[:,1:].values.tolist()

# unique_h3s = set(stop_h3_maps[idx].values())
unique_h3s = [item for sublist in stop_h3_maps for item in sublist.values()]
h3_map = {}
h3_queue = []
for index in unique_h3s:
    h3_queue.append(index)
    res = result_df.loc[result_df['h3'] == index]
    if res.shape[0] > 1:
        res = add_lists(res['frequency'].values.tolist())
        h3_map[index] = res.tolist()
    elif res['frequency'].any():
        h3_map[index] = np.nan_to_num(res['frequency'].values[0]).tolist()
    

### Identifying stop types

For each stop we identify which type of stop it is as specified in the GTFS specifications. We similarly find that custom stop types have been added and map these to a relevant stop type. As thus we encode stop types as follows
* busses (stop code 3), Midttrafik busses (stop code 700 and 715) $\to$ 0
* trams and ferries (stop code 0 and 4) $\to$ 1
* S-train and metro (stop code 109 and 1) $\to$ 2
* Trains (stop code 1) $\to$ 3 

In [13]:
stop_type_map = {}


for _, route in feeds[0].routes.iterrows():
    route_type = route['route_type']
    # s tog becomes metro
    if route_type == 109 or route_type == 1:
        route_type = 2
    # busses in midtrafik
    elif route_type == 700 or route_type == 715 or route_type == 3:
        route_type = 0
    elif route_type == 2:
        route_type = 3
    else:
        route_type = 1

    stops = gk.get_stops(feeds[0], route_ids=[route['route_id']])['stop_id']
    for stop in stops:
        h3_index = stop_h3_maps[0][stop]
        try:
            current = stop_type_map[h3_index]
            if route_type > current:
                stop_type_map[h3_index] = route_type
        except KeyError:
            stop_type_map[h3_index] = route_type

In [14]:
with open("data/stop_type_map.json", "w") as outfile:
    json.dump(stop_type_map, outfile)
with open("data/stop_h3_map.json", "w") as outfile:
    json.dump(stop_h3_maps[0], outfile)

### Read in data from osm_convert.py

The script filters through the danish OSM data keeping only way elements containing the `highway` key and any of the following values
* "footway"
* "path"
* "pedestrian"
* "cycleway"
* "residential"
* "unclassified"
* "tertiary"
* "secondary"
We chose the above keys because of the scope of our project which is determined to cover transit accesibility for all of Denmark. As such, while tertiary and secondary roads might not be accesible by foot because there is no pavement we include them to accomodate the fact that for rural areas these types of stops do exist. For definitions see https://wiki.openstreetmap.org/wiki/Key:highway  
Finally it removes the key value pair `foot:no` see https://wiki.openstreetmap.org/wiki/Key:foot 

The `osm_convert.py` script is too memory intensive to execute through a jupyter notebook, you will have to run this separately. To run this you will also need download the danish OSM data from the following link https://download.geofabrik.de/europe/denmark.html and put it in the ``notebooks`` directory

In [27]:

with open('data/dkall.geojson') as json_file:
    data = json.load(json_file)

# Switching coordinates
for geo in data['features']:
    geo['geometry']['coordinates'] =  [[y,x] for x,y in geo['geometry']['coordinates']]


# Methods to convert OSM ways to H3 hexagons

First we define a function that can project different types of geometries to and from WGS 84 and the UTM zone 32N. We do this to work with buffering in metres. If we buffer using WGS 84 we are more likely to produce distortions when buffering an object.

In [28]:
from pyproj import Transformer, CRS


crs_4326 = CRS.from_epsg(4326)
crs_25831 = CRS.from_epsg(25832)

to_dk = Transformer.from_crs(crs_4326, crs_25831)
to_WGS = Transformer.from_crs(crs_25831, crs_4326)


def CRS_transform(shp, inv=False):
    geoInterface = shp.__geo_interface__

    shpType = geoInterface['type']
    coords = geoInterface['coordinates']
    if shpType == 'Polygon':
        if inv:
            newCoord = [[to_WGS.transform(*point) for point in linring] for linring in coords]
        else:
            newCoord = [[to_dk.transform(*point) for point in linring] for linring in coords]
    elif shpType == 'LineString':
        if inv:
            newCoord = [to_WGS.transform(*point) for point in coords]
        else:
            newCoord = [to_dk.transform(*point) for point in coords]
    elif shpType == 'MultiPolygon':
        if inv:
            newCoord = [[[to_WGS.transform(*point) for point in linring] for linring in poly] for poly in coords]
        else:
            newCoord = [[[to_dk.transform(*point) for point in linring] for linring in poly] for poly in coords]

    return shape({'type': shpType, 'coordinates': tuple(newCoord)})

### Polyfill method

The first method goes through every object and projects the coordinates to the proper CRS with metric units. We then buffer the objects using a 10 meter buffer and reproject back to WGS 84. Then we invoke the H3 polyfill function which transforms a polygon into h3 indices. This is slow 

In [29]:
# from shapely.geometry import mapping, shape


# h3_set = set()
# for geo in data['features']:
#     tmp = shape(geo['geometry'])
#     tmp = CRS_transform(tmp)
#     tmp = tmp.buffer(10)
#     tmp = CRS_transform(tmp, inv = True)

#     if mapping(tmp)['type'] == 'Polygon':
#         h3_set.update(h3.polyfill(mapping(tmp), 11, geo_json_conformant=False))
#     elif mapping(tmp)['type'] == 'MultiPolygon':
#         polygons = list(tmp)
#         for poly in polygons:
#             h3_set.update(h3.polyfill(mapping(poly), 11, geo_json_conformant=False))
#     else:
#         break
    

### Line interpolation method

For this method we reproject the gemetries from the WGS 84 CRS to the UTM zone 32N with metric units. We then create points along the way objects for every 10 meters using shapely interpolation. The point coordinates can then be used to make h3 indices.

In [30]:
# from shapely.geometry import mapping, shape, LineString
# import numpy as np


# h3_set = set()
# for geo in data['features']:
#     tmp = shape(geo['geometry'])
#     tmp = CRS_transform(tmp)
#     distances = np.arange(0, tmp.length, 10)
#     points = [tmp.interpolate(distance) for distance in distances] 
#     if len(points) == 1:
#         continue
#     tmp = LineString(points)
#     tmp = CRS_transform(tmp, inv = True)
#     h3s = {h3.geo_to_h3(lat, lon, 11) for (lat, lon) in mapping(tmp)['coordinates']}
#     h3_set.update(h3s)
    

### H3 line method

The method we ended up using was to use the h3 line function which returns the indices of the hexagons forming a line between two points. We could therefore iterate over the points in every way object from OSM data and generate our walkable hexagons that way.

In [31]:

h3_set = set()
for geo in data['features']:
    tmp = shape(geo['geometry'])
    if len(mapping(tmp)['coordinates']) == 1:
        h3_set.add(h3.geo_to_h3(*mapping(tmp)['coordinates'], resolution = 11))
    for i in range(len(mapping(tmp)['coordinates']) - 1):
        h3s = h3.h3_line( h3.geo_to_h3(*mapping(tmp)['coordinates'][i], resolution = 11) , h3.geo_to_h3(*mapping(tmp)['coordinates'][i+1], resolution = 11) )
        h3_set.update(h3s)

We write this data

In [32]:
walk = pd.DataFrame(list(h3_set), columns=['h3'])
walk.to_csv("data/line_paths_11.csv")

### H3 edges method

This method is similar to the h3 line function described above but only returns the unidirectional edges of the hexagons making up our paths. It could however be used to model flows in future work

In [33]:
h3_set = set()
for geo in data['features']:
    tmp = shape(geo['geometry'])
    if len(mapping(tmp)['coordinates']) == 1:
        continue
        h3_set.add(h3.geo_to_h3(*mapping(tmp)['coordinates'], resolution = 11))
    for i in range(len(mapping(tmp)['coordinates']) - 1):
        h3s = h3.h3_line( h3.geo_to_h3(*mapping(tmp)['coordinates'][i], resolution = 11) , h3.geo_to_h3(*mapping(tmp)['coordinates'][i+1], resolution = 11) )
        h3_edges = [h3.get_h3_unidirectional_edge(h3s[i], h3s[i+1]) for i in range(len(h3s) -1)]
        h3_set.update(h3_edges)


In [34]:
#walk = pd.DataFrame(list(h3_set), columns=['h3'])
#walk.to_csv("data/edges_paths_11.csv")

### Add walking network

In [35]:
walk = pd.read_csv("data/line_paths_11.csv")

We check whether the walkable h3 indices are part of the h3 stop indices and assign them an array of zero frequencies if they are not. 

In [36]:
walk_map = {}

for w in walk['h3']:
    try:
        h3_map[w]
        continue
    except KeyError:
        walk_map[w] = [0]*(24*7)

### Adding the walking network to the stops data

Starting from our stops we go through neighboring hexagons and check whether they are part of the walkable h3 indices. If so, we add them and their frequencies to our h3 map data we then add them to queue and keep track of which hexagons we have visited. By doing this we add our walking network as well as dropping indices that are not connected to a stop.

In [37]:
visited_map = {}

while h3_queue:
    current = h3_queue.pop()
    visited_map[current] = True
    for hexagon in h3.k_ring(current, 1):
        # add hexagon to the network
        try:
            h3_map[hexagon] = walk_map[hexagon]
        except KeyError:
            continue
        # add unvisited hexagons to the queue
        try:
            visited_map[hexagon]
            continue
        except KeyError:
            h3_queue.append(hexagon)



Let's save a checkpoint in case we run into ram issues

In [39]:
with open("data/h3_map.json", "w") as outfile:
    json.dump(h3_map, outfile)

It might be wise to do free memory as you will be needing at least 16 GB later

In [ ]:
%reset
import gc
gc.collect()

# Reading EU urbanization data

We work with raster image data and utilize the rasterio library to handle the layered nature of the data.

In [1]:
import rasterio
import matplotlib.pyplot as plt
from matplotlib import colors
from rasterio.plot import show
import rasterio.warp
from rasterio.crs import CRS
from rasterio.enums import Resampling
import sys
from pathlib import Path
import json
import h3
import pandas as pd
import numpy as np
import geopandas as gp
import shapely as shp
import gtfs_kit as gk
from shapely.geometry import mapping, shape


Here is the data visualized. Every pixel value corresponds to a label of a $1km^2$ square. The data is documented here https://ghsl.jrc.ec.europa.eu/documents/GHSL_Data_Package_2019.pdf pages 17-27

In [ ]:
fp = 'data/GHS_SMOD_POP2015_GLOBE_R2019A_54009_1K_V2_0_18_2.tif'
src = rasterio.open(fp)
fig, ax = plt.subplots(1, figsize=(12, 12))


show(src, ax =ax)

The above picture clearly omits Bornholm. The reason is that the data is downloaded on a per grid level. We therefore need to include the grid containing Bornholm.

In the following code we go through both grids containing the danish data. We start by upsampling the resolution of the images to a 10 times higher resolution and interpolate image values based on bilinear interpolation. This is done to avoid geographical inconsistencies when assigning values to H3 hexagons across larger differences in resolution. We construct arrays of the the coordinate values of the upsampled image pixels, as specified by the CRS of the data which is "World_Mollweide". We transform the coordinates to WGS 84, create the h3 indices at resolution 9 otherwise our map will be incomplete. We specify urban areas as pixels with values 21, 22, 23 or 30 and values 11, 12, 13 as rural areas as specified in the data documentation. Pixels with value 10 are water pixels and they are removed. Finally, we assign hexagons at resolution 11 the corresponding urban-rural label depending on their parent h3 index. If a parent h3 index is from a water pixel we assign it as a rural hexagon.  

`Requires quite a good amount of ram (16GB) and time` If you run into RAM issues, data should be importable through data checkpoints made in the notebook

In [ ]:
fps = ['data/GHS_SMOD_POP2015_GLOBE_R2019A_54009_1K_V2_0_18_2.tif','data/GHS_SMOD_POP2015_GLOBE_R2019A_54009_1K_V2_0_19_2.tif']
urban_map = dict()
for i, fp in enumerate(fps):
    
    with rasterio.open(fp) as src:
        b = src.read(
        out_shape=(
            src.count,
            int(src.height * 10),
            int(src.width * 10)
        ), resampling=Resampling.bilinear)

        transform = src.transform * src.transform.scale(
                src.height / b.shape[-1],
                src.width / b.shape[-2]
            )

        print('Band{i} has shape', b.shape)
        b, trans = rasterio.warp.reproject(
            source = b,
            src_transform= transform,
            src_crs = CRS.from_wkt('PROJCS["World_Mollweide",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],\
                AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["central_meridian",0],\
                PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'),
            dst_crs = CRS.from_epsg(4326)
        )

        height = b.shape[-2]
        width = b.shape[-1]
        ys, xs = np.meshgrid(np.arange(width), np.arange(height))
        xs, ys = rasterio.transform.xy(trans, xs, ys)
        xs= np.array(xs).flatten()
        ys = np.array(ys).flatten()
        for x,y, val in zip(ys, xs, b.flatten()):
            if val == 10 or val == 0:
                continue
            elif val == 21 or val == 22 or val == 23 or val == 30: #urban pixel values
                urban_map[h3.geo_to_h3(x, y, resolution=9)] = 1
            else:
                urban_map[h3.geo_to_h3(x,y, resolution=9)] = 0
with open(f"data/urban.json", "w") as outfile:
    json.dump(urban_map, outfile)


       

Read in data as neccessary

In [2]:
with open('data/h3_map.json') as json_file:
    h3_map = json.load(json_file)


In [ ]:
with open('data/urban.json') as json_file:
    urban_map = json.load(json_file)

In [4]:
for hexagon in h3_map.keys():
    par = h3.h3_to_parent(hexagon, 9)
    try: # some parents are water
        h3_map[hexagon] = (h3_map[hexagon], urban_map[par]) 
    except:
        h3_map[hexagon] = (h3_map[hexagon], 0)

### Export Data

In [5]:
with open("data/h3_map.json", "w") as outfile:
    json.dump(h3_map, outfile)

Read in data if neccesary

In [6]:
with open('data/stop_type_map.json') as json_file:
    stop_type_map = json.load(json_file)

In [23]:
with open('data/h3_map.json') as json_file:
    h3_map = json.load(json_file)

## Final dataset creation

Here we put together the final version of our raw dataset containing h3 indices at resolution 11 for computation, at resolution 10 for aggregation and visualization, and at resolution 4 for fast rendering in visualization. The every index at resolution 11 has an array of length 24*7 containing the frequency of departures at every hour of the week at stops, while for paths all values are 0. The indices also have information about the whether it is urban or rural and what type of stop the index is, as specified by our stop map. For non-stops this value is -1.

In [ ]:
new = pd.DataFrame()
new['h3'] = h3_map.keys()
new['h3_group'] = new.apply(lambda row: h3.h3_to_parent(h = row['h3'], res=4), axis=1)
new['h3_group_agg'] = new.apply(lambda row: h3.h3_to_parent(h = row['h3'], res=10), axis=1)
new['freq'] = [x[0] for x in h3_map.values()]
new['urban'] = [x[1] for x in h3_map.values()]
new['type'] = new.apply(lambda row: -1 if sum(row['freq']) == 0 else stop_type_map[row['h3']], axis=1)
new.head()

In [8]:
new.to_json("data/dataframe.json", orient = 'records')

# Data analysis of spatial container sizes

We did a bit of data analysis on the size of danish spatial containers to determine an appropriate scaling factor of avaiability between rural and urban areas. The spatial containers represent a conceptualization of the hierarchical nature of human cognitive maps. The methodology of computing these spatial containers as well as an introduction to the data can be found here: "The scales of human mobility" (Alessandreti, et. al, 2020) https://www.nature.com/articles/s41586-020-2909-1 

The data used here is derived data and can be found at https://data.dtu.dk/articles/dataset/The_Scales_of_Human_Mobility/12941993/1

In [18]:
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler

We read in the data and extract the sizes of the level 2 container sizes

In [ ]:
dtu = pd.read_pickle('data/Figure_3.pkl')
dtu = dtu[dtu['NAME_0']=='Denmark']
dtu = dtu[dtu.sizes.apply(lambda x: len(x) != 0)]


l_2 = []
for sizes in dtu['sizes']:
    l_2.append(sizes[0])
dtu['l_2'] = l_2

dtu.head()

The data has very long tails (see below) so we either need to handle some of the extreme cases or look at the data on a log scale.

In [ ]:
import joypy

fig, axes = joypy.joyplot(dtu, by="n_scales", column="l_2", figsize=(8,6),
                          title="Container sizes", kind = 'kde')

Implementing a couple of function to remove "outliers"

In [21]:
def is_outlier(s):
    lower_limit = s.mean() - (s.std() * 2)
    upper_limit = s.mean() + (s.std() * 2)
    return ~s.between(lower_limit, upper_limit)

def is_windsor_outlier(s):
    lower_limit, upper_limit = s.quantile([0.05, 0.5])
    return ~s.le(upper_limit) 

def iqr_50(s):
    lower_limit, upper_limit = s.quantile([0.25, 0.75])
    return ~s.between(lower_limit, upper_limit)


Descriptive statistics for urban data, we group by the number of scales each individual hasas this influences their container sizes.

In [ ]:
urban = dtu[(dtu.urban_rural == 'urban')] # (~dtu.n_scales.isin([2,3])
urban['log_l_2'] = urban['l_2'].apply(lambda x: np.log(x))
urban_f = urban[~urban.groupby('n_scales')['l_2'].apply(is_windsor_outlier)]
urban.groupby('n_scales').describe()

In [ ]:
fig, axes = joypy.joyplot(urban, by="n_scales", column="log_l_2", figsize=(8,6),
                          title="Urban container sizes, log scale")

The first thing to notice is that even though these container sizes should represent an area in which each location within the area is considered close. However the mean of each container is more than 2 km with an even larger standard deviation. On the log scale the data might make a bit more sense, however it is immediately apparent that we cannot use container sizes as a measure of what is a good distance to public transport especially when considering accesibility by foot.

What we might try to do is to see if we can determine a scaling factor between the rural and urban sizes which we can use for our accesibility calculations. First we try fitting a Z-score standardizer to the urban container sizes and then transform the rural scales using the urban standardizer. This might give us a better sense of much larger rural containers are than urban. One limitation though is that the rural population has more levels than the urban though this only affects 3 individuals here.

In [ ]:
urban_scales = {}
for i, scale in urban.groupby("n_scales"):
    urban_scales[i] = StandardScaler().fit(X = scale[['log_l_2','l_2']])
#urban[['log_l_2_s','l_2_s']] = urban_scale.transform(urban[['log_l_2','l_2']])

In [ ]:
rural = dtu[(dtu.urban_rural == 'rural')] # & (~dtu.n_scales.isin([2,3]))
rural['log_l_2'] = rural['l_2'].apply(lambda x: np.log(x))
rural_f = rural[~rural.groupby('n_scales')['l_2'].apply(is_windsor_outlier)]
rural.groupby('n_scales').describe()

In [ ]:
fig, axes = joypy.joyplot(rural, by="n_scales", column="log_l_2", figsize=(8,6),
                          title="Rural container sizes, log scale")

We compute the weighted average rural container size expressed as Z-scores from the scaler fitted on urban data  

In [ ]:
rural_resc = pd.DataFrame()
for i, scale in rural.groupby('n_scales'):
    try:
        scale[['log_l_2_s','l_2_s']] = urban_scales[i].transform(scale[['log_l_2','l_2']])
        rural_resc = rural_resc.append(scale)
    except:
        rural_resc = rural_resc.append(scale)



rural_res = rural_resc[['log_l_2_s','l_2_s', 'n_scales']].groupby('n_scales').describe()
rural_res

In [ ]:
(rural_res.reset_index()['log_l_2_s']['mean'] * rural_res.reset_index()['log_l_2_s']['count'] / rural_res.reset_index()['log_l_2_s']['count'].sum()).sum()

We see that the size in rural areas on average is 0.5 standard deviation larger than the mean of cities. However since the we are working with logarithms it is difficult to translate this into actual distances that we can use for scaling since it will depend the initial mean value. 

We also try windzorizing the non-transformed sizes to arrive at at scale in meters that makes sense to consider in relation to transit. We then try to fit mixture model of two gaussians to both rural and urban data.

First the urban data

In [ ]:

gmm = GaussianMixture(
            n_components=2, covariance_type='full', random_state=0
        )
gmm.fit(np.expand_dims(urban_f['l_2'], 1))
Gaussian_nr = 1
print(f"Results for all scales")
for mu, sd, p in zip(gmm.means_.flatten(), np.sqrt(gmm.covariances_.flatten()), gmm.weights_):
    print(f'Normal_distb {Gaussian_nr}: μ = {mu}, σ = {sd}, weight = {p}')
    Gaussian_nr += 1 

for scale, scales_group in urban_f.groupby('n_scales'):
    gmm = GaussianMixture(
            n_components=2, covariance_type='full', random_state=0
        )
    gmm.fit(np.expand_dims(scales_group['l_2'], 1))
    Gaussian_nr = 1
    print(f"Results for {scale} scales")
    for mu, sd, p in zip(gmm.means_.flatten(), np.sqrt(gmm.covariances_.flatten()), gmm.weights_):
        print(f'Normal_distb {Gaussian_nr}: μ = {mu}, σ = {sd}, weight = {p}')
        Gaussian_nr += 1 

Rural data

In [ ]:
gmm = GaussianMixture(
            n_components=2, covariance_type='full', random_state=0
        )
gmm.fit(np.expand_dims(rural_f['l_2'], 1))
Gaussian_nr = 1
print(f"Results for all scales")
for mu, sd, p in zip(gmm.means_.flatten(), np.sqrt(gmm.covariances_.flatten()), gmm.weights_):
    print(f'Normal_distb {Gaussian_nr}: μ = {mu}, σ = {sd}, weight = {p}')
    Gaussian_nr += 1 

for scale, scales_group in rural_f.groupby('n_scales'):
    gmm = GaussianMixture(
            n_components=2, covariance_type='full', random_state=0
        )
    gmm.fit(np.expand_dims(scales_group['l_2'], 1))
    Gaussian_nr = 1
    print(f"Results for {scale} scales")
    for mu, sd, p in zip(gmm.means_.flatten(), np.sqrt(gmm.covariances_.flatten()), gmm.weights_):
        print(f'Normal_distb {Gaussian_nr}: μ = {mu}, σ = {sd}, weight = {p}')
        Gaussian_nr += 1 

We end taking inspiration from the work here when determining the scaling factor for the urban rural distinction but don't choose a specific empirically derived scale.